<a href="https://colab.research.google.com/github/AmrNawar/AIE314-Tutorial1/blob/main/Tutorial1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spire.doc
from spire.doc import *
from spire.doc.common import *

document = Document()
document.LoadFromFile("file-sample_100kB.doc")
document_text = document.GetText()
with open("Output/DocumentText.txt", "w", encoding="utf-8") as file:
    file.write(document_text)

document.Close()

In [2]:
!pip install Spire.Presentation
from spire.presentation import *
from spire.presentation.common import *

presentation = Presentation()
presentation.LoadFromFile("Little Red Riding Hood Storybook by Slidesgo.pptx")

text = []
for slide in presentation.Slides:
    for shape in slide.Shapes:
        if isinstance(shape, IAutoShape) and shape.TextFrame is not None:
            for paragraph in shape.TextFrame.Paragraphs:
                text.append(paragraph.Text)
f = open("Output/ExtractAllText.txt","w", encoding = 'utf-8')
for s in text:
    f.write(s + "\n")
f.close()

# Dispose resources
presentation.Dispose()

In [5]:
!pip install Spire.PDF
from spire.pdf.common import *
from spire.pdf import *
def extract_text_from_pdf(file_path, output_file):
    # Load a PDF document
    doc = PdfDocument()
    # Use the file_path parameter instead of a hardcoded path
    doc.LoadFromFile("sample.pdf")

    extracted_text = []
    for i in range(doc.Pages.Count):
        page = doc.Pages.get_Item(i)
        # Extract the text from the page
        textExtractor = PdfTextExtractor(page)
        option = PdfTextExtractOptions()
        text = textExtractor.ExtractText(option)
        extracted_text.append(text)

    with open(output_file, "w", encoding="utf-8") as text_file:
        text_file.write("\n".join(extracted_text))

    doc.Close()
file_path = "sample-1.pdf"
output_file = "PDFText.txt"
extract_text_from_pdf(file_path, output_file)



In [6]:
!pip install requests beautifulsoup4
import requests
from bs4 import BeautifulSoup

url = "https://www.yallakora.com/egyptian-league/2935/fixtures/%d8%a7%d9%84%d8%af%d9%88%d8%b1%d9%8a-%d8%a7%d9%84%d9%85%d8%b5%d8%b1%d9%8a#tour-matches"
response = requests.get(url)
html_content = response.content

soup = BeautifulSoup(html_content, "html.parser")

# Find all match elements
match_elements = soup.find_all("div", class_="matchCard")

# Extract data from each match element
for match_element in match_elements:
    # Teams
    teams = match_element.find_all("div", class_="teamName")
    team1 = teams[0].text.strip()
    team2 = teams[1].text.strip()

    # Date and Time
    date_time = match_element.find("div", class_="date").text.strip()

    # Stadium (if available)
    stadium = match_element.find("div", class_="stadium").text.strip() if match_element.find("div", class_="stadium") else "N/A"

    # Week (round)
    week = match_element.find("div", class_="week").text.strip()

    # Print the extracted data
    print(f"Teams: {team1} vs {team2}")
    print(f"Date and Time: {date_time}")
    print(f"Stadium: {stadium}")
    print(f"Week: {week}")
    print("-" * 20)

In [7]:
def preprocess_document(file_path):
  if file_path.endswith(' .pdf '):
    text = extract_text_from_pdf(file_path)
    file_type = "PDF"
  elif file_path.endswith(' .docx '):
    text = extract_text_from_docx(file_path)
    file_type = "Word"
  elif file_path.endswith(' .pptx '):
    text = extract_text_from_ppt(file_path)
    file_type = "PowerPoint"
  else:
    raise ValueError("Unsupported File Type")
  metdata = {"author": "Unkown", "date": "2023-10-01", "source": file_path}
  return normalize_to_json(file_type, text, metadata)
  procressed_data = preprocess_document('sample-1.pdf')
  print(procressed_data)

In [10]:
import json
from spire.pdf import PdfDocument, PdfTextExtractor, PdfTextExtractOptions
import os

def extract_text_from_pdf(file_path):
    # Get the absolute path of the file
    file_path = os.path.abspath(file_path)

    doc = PdfDocument()
    doc.LoadFromFile("sample.pdf")
    text = ""

    for i in range(doc.Pages.Count):
        page = doc.Pages.get_Item(i)
        text += PdfTextExtractor(page).ExtractText(PdfTextExtractOptions())
    doc.Close()
    return text

def normalize_to_json(file_type, content, metadata):
    return {"file_type": file_type, "content": content, "metadata": metadata}

# Get the absolute path of the PDF file
pdf_file_path = os.path.abspath("sample-1.pdf")

pdf_text = extract_text_from_pdf(pdf_file_path)
pdf_metadata = {"author": "Unknown", "date": "2023-10-01", "source": pdf_file_path}
pdf_json = normalize_to_json("PDF", pdf_text, pdf_metadata)
print(json.dumps(pdf_json, indent=4))

{
    "file_type": "PDF",
    "content": "Evaluation Warning : The document was created with Spire.PDF for Python.              Sample PDF\n              This is a simple PDF \ufb01le. Fun fun fun.\n\r\n              Lorem ipsum dolor sit amet,  consectetuer adipiscing elit. Phasellus facilisis odio sed mi. \n              Curabitur suscipit. Nullam vel nisi. Etiam semper ipsum ut lectus. Proin aliquam, erat eget \n              pharetra  commodo,  eros  mi  condimentum  quam,  sed  commodo  justo  quam  ut  velit. \n              Integer  a  erat.  Cras  laoreet  ligula  cursus  enim.  Aenean  scelerisque  velit  et  tellus. \n              Vestibulum dictum aliquet sem.  Nulla facilisi.  Vestibulum accumsan  ante  vitae  elit.  Nulla \n              erat  dolor,  blandit  in,  rutrum  quis,  semper  pulvinar,  enim.  Nullam varius  congue  risus. \n              Vivamus  sollicitudin,  metus  ut  interdum  eleifend,  nisi  tellus  pellentesque  elit,  tristique \n              accums

In [14]:
from transformers import pipeline
qa_pipeline = pipeline("question-answering")

def preprocess_document(file_path):
  if file_path.endswith('.pdf'):
    text = extract_text_from_pdf(file_path)
    file_type = "PDF"
  elif file_path.endswith('.docx'):
    text = extract_text_from_docx(file_path)
    file_type = "Word"
  elif file_path.endswith('.pptx'):
    text = extract_text_from_ppt(file_path)
    file_type = "PowerPoint"
  else:
    raise ValueError("Unsupported File Type")
  metadata = {"author": "Unkown", "date": "2023-10-01", "source": file_path}
  return normalize_to_json(file_type, text, metadata)


processed_data = preprocess_document('sample-1.pdf')
print(processed_data)

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


{'file_type': 'PDF', 'content': 'Evaluation Warning : The document was created with Spire.PDF for Python.              Sample PDF\n              This is a simple PDF ﬁle. Fun fun fun.\n\r\n              Lorem ipsum dolor sit amet,  consectetuer adipiscing elit. Phasellus facilisis odio sed mi. \n              Curabitur suscipit. Nullam vel nisi. Etiam semper ipsum ut lectus. Proin aliquam, erat eget \n              pharetra  commodo,  eros  mi  condimentum  quam,  sed  commodo  justo  quam  ut  velit. \n              Integer  a  erat.  Cras  laoreet  ligula  cursus  enim.  Aenean  scelerisque  velit  et  tellus. \n              Vestibulum dictum aliquet sem.  Nulla facilisi.  Vestibulum accumsan  ante  vitae  elit.  Nulla \n              erat  dolor,  blandit  in,  rutrum  quis,  semper  pulvinar,  enim.  Nullam varius  congue  risus. \n              Vivamus  sollicitudin,  metus  ut  interdum  eleifend,  nisi  tellus  pellentesque  elit,  tristique \n              accumsan  eros  quam

In [18]:
import os

# Create the 'output' folder if it doesn't exist
if not os.path.exists('output'):
    os.makedirs('output')



In [19]:
import json
import os

# ... (Your existing code to extract and normalize data) ...

# Assuming your normalization logic is in a function called 'preprocess_document'
# and you want to process 'sample-1.pdf'
normalized_data = preprocess_document('sample-1.pdf')

# Save the normalized JSON data to a file in the 'output' folder
with open(os.path.join('output', 'document_name.json'), 'w', encoding='utf-8') as f:
    json.dump(normalized_data, f, indent=4)